# Scraping: http://www.bbc.co.uk/news

Πάμε να πάρουμε περιεχόμενο από την αρχική σελίδα του BBC News. Θέλουμε:

* Τίτλους (Headlines)
* Συνόψεις (Summary)
* Υπερσυνδέσεις (link)

## Ξεκινώντας

Θέλουμε να **εισάγουμε τα απαραίτητα libraries**.

In [1]:
import requests
from bs4 import BeautifulSoup

Προχωράμε να **κατεβάσουμε την ιστοσελίδα** και *να την εισάγουμε στο BeautifulSoup*.

In [2]:
response = requests.get('http://www.bbc.co.uk/news')
doc = BeautifulSoup(response.text, 'html.parser')

Συνήθως την σελίδα που αναλύουμε στο Beautiful Soup την ονομάζουμε `soup`, αλλά σήμερα θα κάνουμε τη διαφορά και θα την ονομάσουμε `doc`, για να θυμόμαστε ότι είναι ολόκληρο το *document*.

## Προσπάθεια 1η: Παίρνουμε απευθείας τα tags 

Αν κοιτάξουμε την σελίδα, προσπαθούμε με το βελάκι να επιλέξουμε τα headlines, αλλά δεν **γίνεται**. Δεν μπορούμε να επιλέξουμε κανέναν τίτλο! Προφανώς πρόκειται για ολόκληρο το BLOCK του άρθρου ή κάτι τέτοιο? 

Ευτυχώς γνωρίζουμε λίγο HTML, οπότε μπορούμε να επιλέξουμε από το δεξί πλαίσιο που λέγεται Elements. Πάμε στο `h3` tag, το οποίο ξέρουμε ότι περιέχει τον τίτλο και το περιεχόμενο του. 

Χμ, άρα μήπως να πάρουμε απλά όλα τα `h3` tags?

In [3]:
headlines = doc.find_all('h3')

for headline in headlines:
    print(headline.text)

'One billion' could get virus worldwide
'One billion' could get virus worldwide
UK sees weekly deaths reach double normal level
Trump 'can't imagine why' disinfectant calls spiked
Olympics 'will not be postponed again'
The families fostering pets during coronavirus
Top NYC coronavirus doctor takes her own life
French police officers suspended over racist slur
HSBC puts 35,000 job cuts on hold 
'No ray of sun' for jailed Salvadorean gang members
Rising toll in care homes leads to record UK deaths 
Argentina halts all flights until 1 September
Rising toll in care homes leads to record UK deaths 
Argentina halts all flights until 1 September
Nigeria to ease lockdowns in major cities
Man dies after clashes at Lebanon economic protest
India cancels order for 'faulty' China test kits
The Uber driver evicted from home and left to die
Coronavirus immunity: Can you catch it twice?
The 'mystery' of India's low Covid-19 death rate
What is behind Nigeria's unexplained deaths?
Why Mexico's factorie

Αρκετά απλό, αλλά? ...λείπει το link και το summary.

Ok, οπότε μπορούμε να δοκιμάσουμε να πάρουμε όλα τα `a` tags, αλλά σίγουρα θα πάρουμε μαζί με αυτά που θέλουμε και αρκετά σκουπίδια που θα περιέχουν `a` tags - όπως π.χ. το περιεχόμενο στο footer της σελίδας κ.λπ. Πάμε να δούμε μήπως μέσα στα `a` tags του άρθρου υπάρχει κάποιο ξεχωριστό class? 

Αν παρατηρήσουμε καλά θα δούμε ότι όντως είναι `class="gs-c-promo-heading nw-o-link-split__anchor gs-o-faux-block-link__overlay-link gel-pica-bold"`. Αυτό δεν είναι μόνο ένα class, είναι **ΠΟΛΛΑ classes**.

* `gs-c-promo-heading`
* `nw-o-link-split__anchor`
* `gs-o-faux-block-link__overlay-link`
* `gel-pica-bold`

Κάπου εδώ αρχίζουμε τις μαντεψιές. Ας δοκιμάσουμε πρώτα το `gs-c-promo-heading` που φαίνεται αρκετά λογικό να είναι το σωστό!

In [4]:
links = doc.find_all('a', { 'class': 'gs-c-promo-heading' })

for link in links:
    print(link.text)

'One billion' could get virus worldwide
'One billion' could get virus worldwide
Trump 'can't imagine why' disinfectant calls spiked
Olympics 'will not be postponed again'
Video 2 minutes 52 secondsThe families fostering pets during coronavirus
Top NYC coronavirus doctor takes her own life
French police officers suspended over racist slur
HSBC puts 35,000 job cuts on hold 
'No ray of sun' for jailed Salvadorean gang members
Rising toll in care homes leads to record UK deaths 
Argentina halts all flights until 1 September
Rising toll in care homes leads to record UK deaths 
Argentina halts all flights until 1 September
Nigeria to ease lockdowns in major cities
Man dies after clashes at Lebanon economic protest
India cancels order for 'faulty' China test kits
The Uber driver evicted from home and left to die
Coronavirus immunity: Can you catch it twice?
The 'mystery' of India's low Covid-19 death rate
What is behind Nigeria's unexplained deaths?
Why Mexico's factories have not shut down
H

Το αποτέλεσμα φαίνεται μια χαρά! Παίρνει το κείμενο του `h3`, γιατί το `h3` βρίσκεται μέσα στο `a` tag, αλλά δεν περιέχει το *link* του URL. Ας ρίξουμε μια ματιά στο `a` tag...

    <a class="gs-c-promo-heading nw-o-link-split__anchor gs-o-faux-block-link__overlay-link gel-pica-bold" href="/news/world-middle-east-39302560">
   
...το URL κρύβεται μέσα στην ιδιότητα `href`. Αν πάρουμε το link, είναι εύκολο να πάρουμε μετά το attribute, απλά χρησιμοποιούμε το`['href']`

In [5]:
links = doc.find_all('a', { 'class': 'gs-c-promo-heading' })

for link in links:
    print(link.text)
    print(link['href'])

'One billion' could get virus worldwide
/news/world-52453640
'One billion' could get virus worldwide
/news/world-52453640
Trump 'can't imagine why' disinfectant calls spiked
/news/world-us-canada-52450692
Olympics 'will not be postponed again'
/sport/olympics/52453435
Video 2 minutes 52 secondsThe families fostering pets during coronavirus
/news/world-52442424
Top NYC coronavirus doctor takes her own life
/news/world-us-canada-52451094
French police officers suspended over racist slur
/news/world-europe-52440987
HSBC puts 35,000 job cuts on hold 
/news/business-52450850
'No ray of sun' for jailed Salvadorean gang members
/news/world-latin-america-52454519
Rising toll in care homes leads to record UK deaths 
/news/health-52455072
Argentina halts all flights until 1 September
/news/world-latin-america-52450654
Rising toll in care homes leads to record UK deaths 
/news/health-52455072
Argentina halts all flights until 1 September
/news/world-latin-america-52450654
Nigeria to ease lockdown

Ωραίο, ε? Απλά τώρα μας μένει ένα τελευταίο πρόβλημα: **δεν έχουμε τα summaries**. Αλλά οκ, μπορούμε να χρησιμοποιήσουμε τον Inspector για να διαλέξουμε ένα...

    <p class="gs-c-promo-summary gel-long-primer gs-u-mt nw-c-promo-summary">Trade and Nato are high on the agenda as the much-anticipated Washington talks begin.</p>

Και πάλι, πρέπει να διαλέξουμε ένα από όλα αυτά, ας δοκιμάσουμε το `gs-c-promo-summary` που φαίνεται καλό.

In [6]:
summaries = doc.find_all('p', { 'class': 'gs-c-promo-summary' })

for summary in summaries:
    print(summary.text)

An international aid group warns that vulnerable countries need urgent help to avoid major outbreaks.
An international aid group warns that vulnerable countries need urgent help to avoid major outbreaks.
The president says he takes no responsibility for a reported spike in calls to emergency hotlines.
The postponed Tokyo Olympics will be "scrapped" if they cannot take place in 2021, says Games chief Yoshiro Mori.
As animal shelters are forced to limit operations, many people have applied to temporarily rehome a pet.
The father of 49-year-old Dr Lorna Breen says: "She tried to do her job and it killed her."
The two officers in Paris were filmed using an extremely offensive word for North Africans.
The bank says it will pause the cuts to avoid leaving staff unable to find work elsewhere.
El Salvador, already criticised by rights groups, cracks down further on imprisoned gang members.
Outbreaks in care homes claiming 2,000 lives a week, but hospital fatalities are going down.
Aviation ind

Ωραία, αλλά **τώρα έχουμε κολλήσει:** δεν υπάρχει τρόπος να συνδυάσουμε και τα headlines και τα links με τα summaries και ακόμη και αν υπήρχε (ίσως με `zip`), δεν θα ήμασταν σίγουροι ότι είναι σωστό το ματσάρισμα. 

Άρα τι κάνουμε τώρα?

## Προσπάθεια 2η: Parent elements

Όταν επιλέγουμε ένα element - ένα link και το κείμενο του, ή μια λίστα με headlines - μας ενδιαφέρει μόνο το element που κοιτάμε. Μερικές φορές όμως, **πρέπει να πάρουμε πολλά διαφορετικά elements την ίδια στιγμή.** Όταν συμβαίνει αυτό πρέπει να δούμε τι κοινό έχουν όλα αυτά μαζί. 

Αν κοιτάξουμε το summary, το link και τον τίτλο, ίσως βρούμε κάτι σαν το παρακάτω. **Είναι γιγάντιο, αλλά τι να κάνουμε αυτό θέλουμε.**

	<div class="gs-c-promo nw-c-promo gs-o-faux-block-link gs-u-pb gs-u-pb+@m gs-c-promo--inline gs-c-promo--stacked@m nw-u-w-auto gs-c-promo--flex" data-entityid="container-top-stories#3">
		<div class="gs-c-promo-image gs-u-display-none gs-u-display-inline-block@xs gel-1/2@xs gel-1/1@m">
			<div class="gs-o-media-island">
				<div class="gs-o-responsive-image gs-o-responsive-image--16by9"><img alt="Yemeni police gather round bodies of Somali refugees (17/03/19)" class="qa-lazyload-image lazyautosizes lazyloaded" data-sizes="auto" data-srcset="https://ichef.bbci.co.uk/live-experience/cps/240/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 240w, https://ichef.bbci.co.uk/live-experience/cps/320/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 320w, https://ichef.bbci.co.uk/live-experience/cps/480/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 480w, https://ichef.bbci.co.uk/live-experience/cps/624/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 624w, https://ichef.bbci.co.uk/live-experience/cps/800/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 800w" data-widths="[240,320,480,624,800]" sizes="177px" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" srcset="https://ichef.bbci.co.uk/live-experience/cps/240/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 240w, https://ichef.bbci.co.uk/live-experience/cps/320/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 320w, https://ichef.bbci.co.uk/live-experience/cps/480/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 480w, https://ichef.bbci.co.uk/live-experience/cps/624/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 624w, https://ichef.bbci.co.uk/live-experience/cps/800/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 800w"></div>
			</div>
		</div>
		<div class="gs-c-promo-body gel-1/2@xs gel-1/1@m gs-u-mt@m">
			<div>
				<a class="gs-c-promo-heading nw-o-link-split__anchor gs-o-faux-block-link__overlay-link gel-pica-bold" href="/news/world-middle-east-39302560">
				<h3 class="gs-c-promo-heading__title gel-pica-bold nw-o-link-split__text">Attack on Yemen migrant boat kills 42</h3></a>
				<p class="gs-c-promo-summary gel-long-primer gs-u-mt nw-c-promo-summary">It is unclear who was behind a helicopter attack which killed 42 refugees and injured 80.</p>
			</div>
			<ul class="gs-o-list-inline gs-o-list-inline--divided gel-brevier gs-u-mt-">
				<li><span class="gs-c-timestamp gs-o-bullet gs-o-bullet- nw-c-timestamp"><span class="gs-o-bullet__icon gel-icon"><svg viewbox="0 0 32 32">
				<polygon points="17,15.4 17,6 15,6 15,16.6 23.8,21.7 24.8,19.9"></polygon>
				<path d="M16,4c6.6,0,12,5.4,12,12c0,6.6-5.4,12-12,12S4,22.6,4,16C4,9.4,9.4,4,16,4 M16,0C7.2,0,0,7.2,0,16c0,8.8,7.2,16,16,16 s16-7.2,16-16C32,7.2,24.8,0,16,0L16,0z"></path></svg></span><time class="gs-o-bullet__text date qa-status-date relative-time" data-datetime="1h" data-seconds="1489768430" data-timestamp-inserted="true" datetime="2017-03-17T16:33:50.000Z">48 minutes ago</time></span></li>
				<li>
					<a aria-label="From Middle East" class="gs-c-section-link gs-c-section-link--truncate nw-c-section-link nw-o-link nw-o-link--no-visited-state" href="/news/world/middle_east"><span aria-hidden="true">Middle East</span></a>
				</li>
			</ul>
		</div>
	</div>

To πάνω πάνω κομμάτι λέγεται **parent element**, όλα τα άλλα elements βρίσκονται μέσα σε αυτό. Αν θέλουμε να το πάρουμε όλο μαζί, τότε πρέπει πρώτα να παίρνουμε τον κάθε γονέα (parent) κάθε *είδηση* και μετά να πάρουμε τα επιμέρους κομμάτια που υπάρχουν μέσα π.χ. headline, links, εικόνες κ.λπ.

Tο κομμάτι του class είναι `class="gs-c-promo nw-c-promo gs-o-faux-block-link gs-u-pb gs-u-pb+@m gs-c-promo--inline gs-c-promo--stacked@m nw-u-w-auto gs-c-promo--flex" data-entityid="container-top-stories#3"`, το οποίο εντάξει όντως είναι κάπως τρομακτικό, αλλά θα δοκιμάσουμε το `gs-c-promo` και ίσως δουλέψει.

In [7]:
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    print(story.text)

'One billion' could get virus worldwideAn international aid group warns that vulnerable countries need urgent help to avoid major outbreaks.1han hour agoWorld'One billion' could get virus worldwideAn international aid group warns that vulnerable countries need urgent help to avoid major outbreaks.1han hour agoWorldRelated content‘Without food, we’ll die inside our homes’Developing world economies hit hard by coronavirusA visual guide to the economic impact
LiveLiveUK sees weekly deaths reach double normal level 4 minutes ago4 minutes ago Do I have hay fever or coronavirus?4mDo I have hay fever or coronavirus?25 minutes ago Scotland to test all over 70s admitted to hospital25mScotland to test all over 70s admitted to hospital31 minutes ago Abuse victims 'don't have to suffer in silence'31mAbuse victims 'don't have to suffer in silence'Live UK sees weekly deaths reach double normal level  4 minutes ago
Trump 'can't imagine why' disinfectant calls spikedThe president says he takes no resp



Προφανώς δεν μπορούμε να χρησιμοποιήσουμε το `.text` γιατί θα πάρει *τα πάντα* που περιέχουν κείμενο, και το headline *και* το summary. Αντιθέτως αυτό που πρέπει να κάνουμε είναι

* ΒΗΜΑ ΕΝΑ: Χρησιμοποιούμε το doc για να πάρουμε την είδηση 
* ΒΗΜΑ ΔΥΟ: Χρησιμοποιούμε την είδηση για να πάρουμε τον τίτλο 
* ΒΗΜΑ ΤΡΙΑ: Χρησιμοποιούμε την είδηση για να πάρουμε τον link
* ΒΗΜΑ ΤΕΣΣΕΡΑ: Χρησιμοποιούμε την είδηση για να πάρουμε την σύνοψη

### ΒΗΜΑ ΕΝΑ: Χρησιμοποιούμε το doc για να πάρουμε την είδηση (story)

In [8]:
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    print("This is a story")

This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story
This is a story


## ΒΗΜΑ ΔΥΟ + ΤΡΙΑ: Χρησιμοποιούμε την είδηση για να πάρουμε τον τίτλο 

Τώρα μπορούμε να κάνουμε ακριβώς το ίδιο για να πάρουμε τον τίτλο και το link, και μετά με το `['href']` να πάρουμε το link του URL.

In [9]:
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    print("THIS IS A STORY")
    headline = story.find('h3')
    print(headline.text)
    link = story.find('a')
    print(link['href'])
    
    
    headlines = doc.find_all('h3')

for headline in headlines:
    print(headline.text)

THIS IS A STORY
'One billion' could get virus worldwide
/news/world-52453640
THIS IS A STORY
UK sees weekly deaths reach double normal level
/news/live/world-52450742
THIS IS A STORY
Trump 'can't imagine why' disinfectant calls spiked
/news/world-us-canada-52450692
THIS IS A STORY
Olympics 'will not be postponed again'
/sport/olympics/52453435
THIS IS A STORY
The families fostering pets during coronavirus
/news/world-52442424
THIS IS A STORY
Top NYC coronavirus doctor takes her own life
/news/world-us-canada-52451094
THIS IS A STORY
French police officers suspended over racist slur
/news/world-europe-52440987
THIS IS A STORY
HSBC puts 35,000 job cuts on hold 
/news/business-52450850
THIS IS A STORY
'No ray of sun' for jailed Salvadorean gang members
/news/world-latin-america-52454519
THIS IS A STORY
Rising toll in care homes leads to record UK deaths 
/news/health-52455072
THIS IS A STORY
Argentina halts all flights until 1 September
/news/world-latin-america-52450654
THIS IS A STORY
R

## ΒΗΜΑ ΤΕΣΣΕΡΑ: Χρησιμοποιούμε την είδηση για να πάρουμε την σύνοψη

Το ίδιο και πάλι! Αυτή τη φορά ψάχνουμε τις παραγράφους `p`.

In [10]:
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    print("THIS IS A STORY")
    headline = story.find('h3')
    print(headline.text)
    link = story.find('a')
    print(link['href'])
    summary = story.find('p')
    print(summary.text)

THIS IS A STORY
'One billion' could get virus worldwide
/news/world-52453640
An international aid group warns that vulnerable countries need urgent help to avoid major outbreaks.
THIS IS A STORY
UK sees weekly deaths reach double normal level
/news/live/world-52450742


AttributeError: 'NoneType' object has no attribute 'text'

### Λείπουν elements

Αυτό μας έλειπε τώρα, ένα error! Συγκεκριμένα λέει ότι ...

    ---------------------------------------------------------------------------
    AttributeError                            Traceback (most recent call last)
    <ipython-input-20-e55795264040> in <module>()
          7     print(link['href'])
          8     summary = story.find('p')
    ----> 9     print(summary.text)

    AttributeError: 'NoneType' object has no attribute 'text'

Εφόσον το λάθος εμφανίστηκε από την στιγμή που βάλαμε το κομμάτι με το `summary` part, υποθέτουμε ότι το πρόβλημα είναι ότι δεν έχει κάθε μια **ιστορία από ένα summary**. Πώς θα το λύσουμε αυτό???!!!

Λοιπόν, μπορούμε να *ρωτάμε αν (if) η ιστορία έχει summary*. Αν έχει τότε μπορούμε να το χρησιμοποιήσουμε. Αν δεν έχει το αγνοούμε. **Είναι ένα απλό `if` statement**.

In [11]:
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    print("THIS IS A STORY")
    headline = story.find('h3')
    print(headline.text)
    link = story.find('a')
    print(link['href'])
    summary = story.find('p')
    if summary:
        print(summary.text)

THIS IS A STORY
'One billion' could get virus worldwide
/news/world-52453640
An international aid group warns that vulnerable countries need urgent help to avoid major outbreaks.
THIS IS A STORY
UK sees weekly deaths reach double normal level
/news/live/world-52450742
THIS IS A STORY
Trump 'can't imagine why' disinfectant calls spiked
/news/world-us-canada-52450692
The president says he takes no responsibility for a reported spike in calls to emergency hotlines.
THIS IS A STORY
Olympics 'will not be postponed again'
/sport/olympics/52453435
The postponed Tokyo Olympics will be "scrapped" if they cannot take place in 2021, says Games chief Yoshiro Mori.
THIS IS A STORY
The families fostering pets during coronavirus
/news/world-52442424
As animal shelters are forced to limit operations, many people have applied to temporarily rehome a pet.
THIS IS A STORY
Top NYC coronavirus doctor takes her own life
/news/world-us-canada-52451094
The father of 49-year-old Dr Lorna Breen says: "She tried

## Ας το σώσουμε ως CSV

Τώρα που έχουμε όλα τα στοιχεία που θέλαμε μπορούμε να τα μετατρέψουμε σε ένα CSV. Υπάρχουν 3 βήματα για τη μετατροπή σε CSV:
    
1. **Ξεκινάμε με μια άδεια λίστα:** Κάθε ιστορία που βρίσκουμε, την προσθέτουμε στη λίστα
2. **Φτιάχνουμε ένα λεξικό** για κάθε μια ιστορία
3. **Μετατρέπουμε τη λίστα σε ένα DataFrame**, μετά
4. **Εξάγουμε το DataFrame σε ένα CSV**

Η δημιουργία του λεξικού μπορεί να είναι λίγο περίπλοκη, οπότε ας δούμε **2 διαφορετικούς τρόπους να το κάνουμε**.

### Μέθοδος 1η: Όλα μαζί

Θα φτιάξουμε το λεξικό `story_dict` απευθείας και μετά θα το προσθέσουμε στη λίστα `stories_list`.

In [12]:
# Ξεκινάμε με μια άδεια λίστα
stories_list = []
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    headline = story.find('h3')
    link = story.find('a')
    summary = story.find('p')
    # Έχει η ιστορία summary?
    if summary:
        summary_text = summary.text
    else:
        summary_text = ''
        
        # Φτιάξε ένα λεξικό αν ΕΧΕΙ summary
    story_dict = {
            'headline': headline.text,
            'url': link['href'],
            'summary': summary_text
        }
     
    # Πρόσθεσε το λέξικο στη λίστα
    stories_list.append(story_dict)

print(stories_list)

# Τώρα που τελειώσαμε μετέτρεψε το σε CSV και αποθήκευσε το.
# Αν δεν χρησιμοποιήσετε το index=False, θα έχετε ένα άσχημο dataframe!
import pandas as pd
df = pd.DataFrame(stories_list)
df.to_csv("bbc.csv", index=False)

[{'headline': "'One billion' could get virus worldwide", 'url': '/news/world-52453640', 'summary': 'An international aid group warns that vulnerable countries need urgent help to avoid major outbreaks.'}, {'headline': 'UK sees weekly deaths reach double normal level', 'url': '/news/live/world-52450742', 'summary': ''}, {'headline': "Trump 'can't imagine why' disinfectant calls spiked", 'url': '/news/world-us-canada-52450692', 'summary': 'The president says he takes no responsibility for a reported spike in calls to emergency hotlines.'}, {'headline': "Olympics 'will not be postponed again'", 'url': '/sport/olympics/52453435', 'summary': 'The postponed Tokyo Olympics will be "scrapped" if they cannot take place in 2021, says Games chief Yoshiro Mori.'}, {'headline': 'The families fostering pets during coronavirus', 'url': '/news/world-52442424', 'summary': 'As animal shelters are forced to limit operations, many people have applied to temporarily rehome a pet.'}, {'headline': 'Top NYC c

### Μέθοδος 2η: Γεμίζοντας τα κενά

Για αυτή τη μέθοδο θα φτιάξουμε πρώτα το λεξικό μας `story_dict` και μετά θα το γεμίζουμε με τα στοιχεία που υπάρχουν. 

In [13]:
# Ξεκινάμε με μια άδεια λίστα
stories_list = []
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    # Φτιάχνουμε ένα λεξικό χωρίς τίποτα
    story_dict = {}
    headline = story.find('h3')
    if headline:
        story_dict['headline'] = headline.text
    link = story.find('a')
    if link:
        story_dict['url'] = link['href']
    summary = story.find('p')
    if summary:
        story_dict['summary'] = summary.text
    # Προσθέτουμε το λεξικό στη λίστα 
    stories_list.append(story_dict)
    
# Τώρα που τελειώσαμε μετέτρεψε το σε CSV και αποθήκευσε το.
# Αν δεν χρησιμοποιήσετε το index=False, θα έχετε ένα άσχημο dataframe!
import pandas as pd
df = pd.DataFrame(stories_list)
df.to_csv("bbc.csv", index=False)